# Tables

## Setup

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path('../data/')


In [2]:
subsets = {
    'PatchFinder_top10': '\\textsc{PatchFinder\\textsubscript{top10}}',
    'random_10': '\\textsc{Random\\textsubscript{10}}',
}

models = {
    "codereviewer": "CodeReviewer",
    "Llama-3.3-70B-Instruct": "Llama-3.3-70B-Instruct",
    "Qwen3-235B-A22B-Instruct-2507": "Qwen3-235B-A22B-Instruct-2507"
}

approaches = {
    'PatchFinder': 'PatchFinder',
    'LLM4VFD': 'LLM4VFD',
    'agent': 'Agent',
}

columns = ['accuracy', 'precision', 'recall', 'f1']

metrics = {
    'accuracy': 'Accuracy',
    'precision': 'Precision',
    'recall': 'Recall',
    'f1': 'F1-score',
}

In [3]:
import os
import pandas as pd


dfs = []
for fname in os.listdir(DATA_DIR):
    if fname.endswith(".csv") and fname.startswith("results_"):
        # Extract the 'xxx' part
        approach = fname.replace("results_", "").replace(".csv", "")
        
        # Read CSV
        df = pd.read_csv(os.path.join(DATA_DIR, fname))
        
        # Add new column
        df["approach"] = approach
        
        dfs.append(df)

# Optionally concatenate them all
data = pd.concat(dfs, ignore_index=True)
data

,subset,model,correct_cves,total_cves,cve_accuracy,accuracy,precision,recall,f1,approach
0,PatchFinder_top10,codereviewer,655,1252,0.523163,0.896959,0.398195,0.367253,0.382098,PatchFinder
1,random_10,codereviewer,244,1252,0.194888,0.879785,0.859719,0.231018,0.364177,PatchFinder
2,PatchFinder_top10,Llama-3.3-70B-Instruct,358,1250,0.286400,0.805962,0.301165,0.941176,0.456315,agent
3,random_10,Llama-3.3-70B-Instruct,682,1252,0.544728,0.934498,0.740161,0.856281,0.793998,agent
4,PatchFinder_top10,Qwen3-235B-A22B-Instruct-2507,553,1250,0.442400,0.866650,0.390458,0.951628,0.553721,agent
5,random_10,Qwen3-235B-A22B-Instruct-2507,852,1252,0.680511,0.952958,0.818600,0.875888,0.846276,agent
6,PatchFinder_top10,Llama-3.3-70B-Instruct,464,1249,0.371497,0.801443,0.288396,0.871628,0.433395,LLM4VFD
7,random_10,Llama-3.3-70B-Instruct,668,1247,0.535686,0.926778,0.762993,0.729656,0.745952,LLM4VFD
8,PatchFinder_top10,Qwen3-235B-A22B-Instruct-2507,465,1249,0.372298,0.798122,0.290830,0.926716,0.442721,LLM4VFD
9,random_10,Qwen3-235B-A22B-Instruct-2507,703,1248,0.563301,0.927257,0.735146,0.793910,0.763399,LLM4VFD


In [4]:
def escape_latex(text):
    """Escape LaTeX special characters."""
    return text.replace('_', '\\_').replace('%', '\\%').replace('&', '\\&').replace('$', '\\$')

## Table 1

In [5]:
table = []

table.append("\\begin{table*}[htbp]")
table.append("\\begin{threeparttable}")
table.append("    \\newcolumntype{Y}{>{\\centering\\arraybackslash}X}")
table.append("    \\newcolumntype{R}{>{\\raggedright\\arraybackslash}X}")
table.append("    \\newcolumntype{L}{>{\\raggedleft\\arraybackslash}X}")
table.append("    \\centering")
#table.append("    \\footnotesize")
table.append("    \\caption{Some caption.}\\label{tab:eval-summary}")

row = "    \\begin{tabularx}{\\textwidth}{lll!{\\color{white}\\hspace{.5em}}"

for i, _ in enumerate(columns, start=1):
    row += "Y"
row += "}"
table.append(row)

table.append("        \\toprule")
header = "        \\textbf{Model} & \\textbf{Subset} & \\textbf{Approach} & "
for col in columns:
    header += f"\\textbf{{{escape_latex(metrics[col])}}} & "
header = header.rstrip(" & ") + " \\\\"
table.append(header)
table.append("        \\midrule")


for i, model in enumerate(data['model'].unique()):
    model_data = data[data['model'] == model]
    num_subsets = model_data['subset'].nunique()
    
    for j, subset in enumerate(subsets.keys()):
        
        subset_data = model_data[model_data['subset'] == subset]
        num_approaches = subset_data['approach'].nunique()
        for k, approach in enumerate(subset_data['approach'].unique()):
            
            if j == 0 and k == 0:
                row = f"        \\multirow{{{num_subsets*num_approaches}}}{{*}}{{{escape_latex(models[model])}}} & "
            else:
                row = "        & "
            if k == 0:
                row += f"\\multirow{{{num_approaches}}}{{*}}{{{escape_latex(subsets[subset])}}} & "
            else:
                row += "        & "
                
            row += f"{escape_latex(approaches[approach])} & "
            
            approach_data = subset_data[subset_data['approach'] == approach]
            if not approach_data.empty:
                for col in columns:
                    value = approach_data[col].values[0]
                    formatted_value = f"{value:.2f}"
                    # add cellcolor to every other row
                    if approach == "agent":
                        formatted_value = f"\\cellcolor{{gray!15}}{formatted_value}"
                    
                    row += f"{formatted_value} & "
                row = row.rstrip(" & ") + " \\\\"
                table.append(row)
            

             
        if j < len(subsets) - 1:
            table.append("        \\cmidrule(lr){2-" + str(3 + len(columns)) + "}")

        
        
    if i < len(data['model'].unique()) - 1:
        table.append("        \\midrule")



table.append("       \\bottomrule")
table.append("    \\end{tabularx}")
table.append("    \\begin{tablenotes}[flushleft]\\small")
table.append("      \\item \\textbf{Bold}: Notes...")
table.append("    \\end{tablenotes}")
table.append("\\end{threeparttable}")
table.append("\\end{table*}")

print("\n".join(table))


\begin{table*}[htbp]
\begin{threeparttable}
    \newcolumntype{Y}{>{\centering\arraybackslash}X}
    \newcolumntype{R}{>{\raggedright\arraybackslash}X}
    \newcolumntype{L}{>{\raggedleft\arraybackslash}X}
    \centering
    \caption{Some caption.}\label{tab:eval-summary}
    \begin{tabularx}{\textwidth}{lll!{\color{white}\hspace{.5em}}YYYY}
        \toprule
        \textbf{Model} & \textbf{Subset} & \textbf{Approach} & \textbf{Accuracy} & \textbf{Precision} & \textbf{Recall} & \textbf{F1-score} \\
        \midrule
        \multirow{2}{*}{CodeReviewer} & \multirow{1}{*}{\textsc{PatchFinder\textsubscript{top10}}} & PatchFinder & 0.90 & 0.40 & 0.37 & 0.38 \\
        \cmidrule(lr){2-7}
        & \multirow{1}{*}{\textsc{Random\textsubscript{10}}} & PatchFinder & 0.88 & 0.86 & 0.23 & 0.36 \\
        \midrule
        \multirow{4}{*}{Llama-3.3-70B-Instruct} & \multirow{2}{*}{\textsc{PatchFinder\textsubscript{top10}}} & Agent & \cellcolor{gray!15}0.81 & \cellcolor{gray!15}0.30 & \cellcolor{gra

In [6]:
result = "\n".join(table)
table_path = Path.cwd().parent / 'tables' / 'eval_summary.tex'
table_path.parent.mkdir(parents=True, exist_ok=True)
with open(table_path, 'w') as f:
    f.write(result)